# Задача

Необходимо написать функцию для построения матрицы миграции просрочки по банкам.	


Функция на вход должна принимать  "Наименование Банка", "Тип обязательства", "Путь к базе данных", т.е. пользователь функции должен иметь возможность построить матрицу миграции по различным банкам и типам обязательств.
На выходе пользователь должен видеть матрицу, выгруженную на новый лист.

Разбивка для матрицы:

- Без просрочки		
- просрочка 0 - 30 дней		
- просрочка 31-60 дней		
- просрочка 61-90 дней		
- просрочка 91+ дней		


In [1]:
import pandas as pd
from openpyxl import load_workbook
import datetime
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [2]:
pip install XlsxWriter

Note: you may need to restart the kernel to use updated packages.


In [3]:
path_read = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSVV3mwtVxLRKfi40fKb_61GPkQtTfRc52TvN4JVVfcp1RuJjfnyKj0-VAOQFCY4mw2JR-8kE7CPvpR/pub?gid=1514326207&single=true&output=csv'
df = pd.read_csv(path_read)
df.head()

,id,Дата,"Просрочка, дней",Тип обязательства,Первоначальный кредитор
0,1001,01/01/2019,0,Ипотека,СБЕРБАНК
1,3828,01/01/2019,2771,Потребительский,ТИНЬКОФФ
2,3829,01/01/2019,2827,Ипотека,ТИНЬКОФФ
3,3831,01/01/2019,3619,Ипотека,ТИНЬКОФФ
4,3832,01/01/2019,3183,Потребительский,ТИНЬКОФФ


In [4]:
def build_migration_matrix(bank_name, obligation_type,\
                           path='D:/Рабочий стол/Наталья/Data analyse/Трудоустройство/Тестовые/РегионКонсалт/Блок3_Данные.csv'):
    #на вход в функцию передаём название банка (первоначальный кредитор, тип обязательства, путь к базе данный 
    #тут указан дефолтный путь,поэтому поменяла местами входные данные
    df = pd.read_csv(path)
    #Если данные их Excel, то использовать этот способ:
    #df = pd.read_excel(path, sheet_name='Data') #(предполагается, что данные находятся в листе "Data" в файле базы данных)
    
    # Фильтрация данных по указанному банку и типу обязательства
    filtered_df = df[(df['Первоначальный кредитор'] == bank_name) & (df['Тип обязательства'] == obligation_type)]
    
    # Разбивка просрочек по временным интервалам
    bins = [-float("inf"), 0, 30, 60, 90, float('inf') ]
    labels = ['Без просрочки', 'просрочка 0-30 дней', 'просрочка 31-60 дней', 'просрочка 61-90 дней', 'просрочка 91+ дней']
    filtered_df["Просрочка категория"] = pd.cut(filtered_df['Просрочка, дней'], bins=bins, labels=labels)
    
    
    # Построение матрицы миграции
    df_date1 = (
        filtered_df.sort_values(by=['id', 'Дата'])
        .groupby('id')
        .agg({'Дата': 'first', 'Просрочка категория': 'first' })
        .rename(columns={'Дата': 'date1', 'Просрочка категория': 'delay1' })
        .reset_index()  
    )
    
    df_date2 = (
        filtered_df.sort_values(by=['id', 'Дата'])
        .groupby('id')
        .agg({'Дата': 'last', 'Просрочка категория': 'last' })
        .rename(columns={'Дата': 'date2', 'Просрочка категория': 'delay2' })
        .reset_index()
    )  
    
    result = pd.merge(df_date2, df_date1, how='outer', on='id')
    result1 = result.pivot_table(index='delay2', columns='delay1', aggfunc='size', fill_value=0).reset_index()
    #Установка необходимых названий строк и столбцов
    result1['Банк'] = bank_name
    result1['Дата актуальная'] = filtered_df['Дата'].max()
    result1.set_index(['Банк', 'Дата актуальная','delay2'], inplace=True)
    date_hist = filtered_df['Дата'].min()
    result1.columns = pd.MultiIndex.from_tuples(
    zip([date_hist, date_hist,date_hist, date_hist, date_hist, date_hist, date_hist, date_hist], 
        result1.columns))
    
    # Загрузка таблицы в файл Excel
    
    with pd.ExcelWriter('Transition matrix.xlsx', engine='xlsxwriter') as writer:
        result1.to_excel(writer, sheet_name='NewSheet', index=True)

        # Получаем объект workbook и worksheet для дополнительной настройки
        workbook  = writer.book
        worksheet = writer.sheets['NewSheet']

        # Задаем ширину для каждого столбца
        col_widths = [20, 20, 20, 20, 20, 20,20,20]
        for i, width in enumerate(col_widths):
            worksheet.set_column(i, i, width)
    #можно указать путь исходного файла, чтобы вторым листом туда записывалось
    #тогда код будет такой:
    #with pd.ExcelWriter(path, engine='openpyxl',mode='a') as writer:
        #writer.book = load_workbook(path)
        #result1.to_excel(writer, sheet_name='Матрица миграции', index=True)
    
    print('Таблица успешно сохранена в Excel')     

In [5]:
build_migration_matrix('ТИНЬКОФФ', 'Потребительский', path_read)

Таблица успешно сохранена в Excel
